In [6]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
PACKAGE_ROOT = os.path.dirname(os.path.abspath(""))
print(PACKAGE_ROOT)
sys.path.insert(0, PACKAGE_ROOT)
import joblib
from commons import constants as C
from time import time, gmtime
import pandas as pd
from sklearn_crfsuite import CRF
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
from utils.utils import get_connl_data, SentenceGetter
# from preprocessing.crf_preprocessor import word2features, sent2features, sent2labels, sent2tokens
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from preprocessing.crf_preprocessor import CRFPreprocessor

c:\Users\sharm\Documents\nlp_assignment\ner-bilstm-crf


In [7]:
train_df = get_connl_data("train")
valid_df = get_connl_data("valid")
test_df = get_connl_data("test")

obj = CRFPreprocessor()
train_sentences = obj.get_sentences(train_df)
valid_sentences = obj.get_sentences(valid_df)
test_sentences = obj.get_sentences(test_df)

X_train, y_train = obj.preprocess(train_sentences)
X_valid, y_valid = obj.preprocess(valid_sentences)
X_test, y_test = obj.preprocess(test_sentences)

In [3]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [7]:
obj.save_model()

['C:\\Users\\sharm\\Documents\\nlp_assignment\\ner-bilstm-crf\\outputs\\crf_model_time.struct_time(tm_year=2023, tm_mon=3, tm_mday=23, tm_hour=16, tm_min=29, tm_sec=41, tm_wday=3, tm_yday=82, tm_isdst=0).pkl']

In [8]:
crf = joblib.load(r"C:\Users\sharm\Documents\nlp_assignment\ner-bilstm-crf\outputs\crf_model_time.struct_time(tm_year=2023, tm_mon=3, tm_mday=23, tm_hour=16, tm_min=29, tm_sec=41, tm_wday=3, tm_yday=82, tm_isdst=0).pkl")
obj.inference(crf, X_train, y_train)

,precision,recall,f1-score,support
B-AerospaceManufacturer,1.000000,0.949438,0.974063,178.0
B-AnatomicalStructure,0.996454,0.872671,0.930464,322.0
B-ArtWork,1.000000,0.906250,0.950820,160.0
B-Artist,0.987044,0.980925,0.983975,2097.0
B-Athlete,0.992391,0.979614,0.985961,932.0
B-CarManufacturer,0.994595,0.920000,0.955844,200.0
B-Cleric,0.995556,0.965517,0.980306,232.0
B-Clothing,0.993007,0.780220,0.873846,182.0
B-Disease,0.986799,0.914373,0.949206,327.0
B-Drink,0.981595,0.919540,0.949555,174.0


In [9]:
obj.inference(crf, X_valid, y_valid)

,precision,recall,f1-score,support
B-AerospaceManufacturer,1.000000,0.700000,0.823529,10.0
B-AnatomicalStructure,0.900000,0.529412,0.666667,17.0
B-ArtWork,0.000000,0.000000,0.000000,9.0
B-Artist,0.619403,0.691667,0.653543,120.0
B-Athlete,0.380952,0.355556,0.367816,45.0
B-CarManufacturer,0.500000,0.333333,0.400000,12.0
B-Cleric,0.500000,0.272727,0.352941,11.0
B-Clothing,0.666667,0.200000,0.307692,10.0
B-Disease,0.800000,0.444444,0.571429,18.0
B-Drink,1.000000,0.666667,0.800000,9.0


In [10]:
obj.inference(crf, X_test, y_test)

,precision,recall,f1-score,support
B-AerospaceManufacturer,0.759259,0.542720,0.632982,831.0
B-AnatomicalStructure,0.868018,0.348828,0.497662,5034.0
B-ArtWork,0.461825,0.261603,0.334007,948.0
B-Artist,0.563151,0.649368,0.603194,31235.0
B-Athlete,0.439694,0.404449,0.421336,13935.0
B-CarManufacturer,0.720191,0.402130,0.516092,2253.0
B-Cleric,0.687310,0.318783,0.435552,3551.0
B-Clothing,0.876147,0.191287,0.314016,1997.0
B-Disease,0.803235,0.424761,0.555674,4911.0
B-Drink,0.736181,0.323936,0.449904,1809.0


In [ ]:
# import eli5
# # eli5.show_weights(crf, top=30)

In [ ]:
# !pip install eli5
# import eli5

In [ ]:
# eli5.show_weights(crf, top=30)